# 2.13

## Model


### Input parameters

Let 
    
\begin{alignat*}{2}
  R & = \text{set of raw materials} = \{ \text{Alloy1, Alloy2, Alloy3, Scrap1, Scrap2} \}\\ 
  C & = \text{set of characteristics} = \{\text{Carbon, Nickel, Chromium, TensileStrength}\}\\
  c_i & = \text{cost of raw material $i$} && \text{for } i \in R\\
  a_i & = \text{amount of raw material $i$ available} && \text{for } i \in R\\
  h_{ij} & = \text{value of characteristic $j$ in raw material $i$} && \text{for } i \in R, j \in C\\
  \ell_j & = \text{lower bound for characteristic $j$} && \text{for } j \in C\\
  u_j & = \text{upper bound for characteristic $j$} && \text{for } j \in C
\end{alignat*}
    
Note that according to the problem description, $\ell_{\text{Nickel}} = 0$.

### Decision variables

Let
  \begin{equation*}
    x_i = \text{tons of raw material $i$ used} \quad \text{for } i \in R
  \end{equation*}

### Objective function and constraints

\begin{alignat*}{3}
\min \quad & \sum_{i \in R} c_i x_i &\quad& &\quad& \text{(total cost)}\\ 
\text{s.t.} \quad & \sum_{i \in R} x_i = 100 &\quad&  &\quad& \text{(piece is 100 tons)}\\
& \ell_j \sum_{i \in R} x_i \leq \sum_{i \in R} h_{ij} x_i \leq u_j \sum_{i \in R} x_i &\quad& \text{for $j \in C$} &\quad& \text{(characteristic requirements)}\\
& 0 \leq x_i \leq a_i &\quad& \text{for $i \in R$} &\quad& \text{(nonnegativity, availability of raw materials)}
\end{alignat*}

### Optimal value

7086.076818

<hr style="height:2px;background-color:gray;">

## Read data

In [1]:
import pandas as pd
print(f'Pandas version: {pd.__version__}')

Pandas version: 1.2.5


In [2]:
# index_col=0 reads the 0th column as the index: this will be
# very useful when converting these DataFrames to dictionaries
#
# We can read all sheets at once with sheet_name=None, 
# but this doesn't generalize if the data is in multiple CSV files
#
# Reading multiple CSV files will work essentially the same way
# with pd.read_csv(), with different filenames and without the
# sheet_name=... keyword argument
R_table = pd.read_excel('2.13.xlsx', sheet_name='R', index_col=0)
R_table

,c,a
R,,
Alloy1,150,50
Alloy2,120,50
Alloy3,80,20
Scrap1,35,30
Scrap2,20,40


In [3]:
R_table['c'].to_dict()

{'Alloy1': 150, 'Alloy2': 120, 'Alloy3': 80, 'Scrap1': 35, 'Scrap2': 20}

In [4]:
C_table = pd.read_excel('2.13.xlsx', sheet_name='C', index_col=0)
C_table

,l,u
C,,
Carbon,0.020,0.030
Nickel,0.000,0.040
Chromium,0.013,0.027
TensileStrength,50000.000,80000.000


In [5]:
h_table = pd.read_excel('2.13.xlsx', sheet_name='h', index_col=0)
h_table

,Carbon,Nickel,Chromium,TensileStrength
Alloy1,0.0175,0.020,0.035,60000
Alloy2,0.0245,0.030,0.008,40000
Alloy3,0.0280,0.040,0.012,90000
Scrap1,0.0310,0.045,0.039,120000
Scrap2,0.0350,0.055,0.028,70000


In [6]:
# Note: DataFrame.stack() unpivots a DataFrame into a Series
# with a multi-level index: the first level contains the original
# row labels, the second level contains the column names
h_table.stack().to_dict()

{('Alloy1', 'Carbon'): 0.0175,
 ('Alloy1', 'Nickel'): 0.02,
 ('Alloy1', 'Chromium'): 0.035,
 ('Alloy1', 'TensileStrength'): 60000.0,
 ('Alloy2', 'Carbon'): 0.0245,
 ('Alloy2', 'Nickel'): 0.03,
 ('Alloy2', 'Chromium'): 0.008,
 ('Alloy2', 'TensileStrength'): 40000.0,
 ('Alloy3', 'Carbon'): 0.028,
 ('Alloy3', 'Nickel'): 0.04,
 ('Alloy3', 'Chromium'): 0.012,
 ('Alloy3', 'TensileStrength'): 90000.0,
 ('Scrap1', 'Carbon'): 0.031,
 ('Scrap1', 'Nickel'): 0.045,
 ('Scrap1', 'Chromium'): 0.039,
 ('Scrap1', 'TensileStrength'): 120000.0,
 ('Scrap2', 'Carbon'): 0.035,
 ('Scrap2', 'Nickel'): 0.055,
 ('Scrap2', 'Chromium'): 0.028,
 ('Scrap2', 'TensileStrength'): 70000.0}

<hr style="height:2px;background-color:gray;">

## Implement model in Pyomo

In [7]:
import pyomo  # just to get version number
import pyomo.environ as pyo
print(f'Pyomo version: {pyomo.__version__}')

Pyomo version: 6.0.1


In [8]:
model = pyo.ConcreteModel()

In [9]:
model.R = pyo.Set(initialize=R_table.index)
model.C = pyo.Set(initialize=C_table.index)

In [10]:
# This is where the Pandas DataFrames get transformed into dictionaries
# so that Pyomo can read them. Some notes:
# - DataFrame['column name'] returns a Series
# - Series.to_dict() returns a dictionary with the index as keys
# - DataFrame.stack() unpivots a table
model.c = pyo.Param(model.R, initialize=R_table['c'].to_dict())
model.a = pyo.Param(model.R, initialize=R_table['a'].to_dict())
model.l = pyo.Param(model.C, initialize=C_table['l'].to_dict())
model.u = pyo.Param(model.C, initialize=C_table['u'].to_dict())
model.h = pyo.Param(model.R, model.C, initialize=h_table.stack().to_dict())

In [11]:
model.x = pyo.Var(model.R, domain=pyo.NonNegativeReals)

\begin{alignat*}{3}
\min \quad & \sum_{i \in R} c_i x_i &\quad& &\quad& \text{(total cost)}
\end{alignat*}

In [12]:
def total_cost_rule(model):
    return sum(model.c[i] * model.x[i] for i in model.R)

model.total_cost = pyo.Objective(
    rule=total_cost_rule,
    sense=pyo.minimize
)

\begin{alignat*}{3}
& \sum_{i \in R} x_i = 100 &\quad&  &\quad& \text{(piece is 100 tons)}\\
\end{alignat*}

In [13]:
def tonnage_rule(model):
    return sum(model.x[i] for i in model.R) == 100

model.tonnage = pyo.Constraint(rule=tonnage_rule)

\begin{alignat*}{3}
& \ell_j \sum_{i \in R} x_i \leq \sum_{i \in R} h_{ij} x_i \leq u_j \sum_{i \in R} x_i &\quad& \text{for $j \in C$} &\quad& \text{(characteristic requirements)}\\
\end{alignat*}

In [14]:
def characteristic_lower_rule(model, j):
    return (
        model.l[j] * sum(model.x[i] for i in model.R)
        <= sum(model.h[i,j] * model.x[i] for i in model.R)
    )

model.characteristic_lower = pyo.Constraint(
    model.C, 
    rule=characteristic_lower_rule
)

In [15]:
def characteristic_upper_rule(model, j):
    return (
        sum(model.h[i,j] * model.x[i] for i in model.R)
        <= model.u[j] * sum(model.x[i] for i in model.R)        
    )

model.characteristic_upper = pyo.Constraint(
    model.C, 
    rule=characteristic_upper_rule
)

\begin{alignat*}{3}
& 0 \leq x_i \leq a_i &\quad& \text{for $i \in R$} &\quad& \text{(nonnegativity, availability of raw materials)}
\end{alignat*}

In [16]:
# Nonnegativity taken care of in variable definition
def bounds_rule(model, i):
    return model.x[i] <= model.a[i]

model.bounds = pyo.Constraint(model.R, rule=bounds_rule)

In [17]:
model.pprint()

3 Set Declarations
    C : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'Carbon', 'Nickel', 'Chromium', 'TensileStrength'}
    R : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {'Alloy1', 'Alloy2', 'Alloy3', 'Scrap1', 'Scrap2'}
    h_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    R*C :   20 : {('Alloy1', 'Carbon'), ('Alloy1', 'Nickel'), ('Alloy1', 'Chromium'), ('Alloy1', 'TensileStrength'), ('Alloy2', 'Carbon'), ('Alloy2', 'Nickel'), ('Alloy2', 'Chromium'), ('Alloy2', 'TensileStrength'), ('Alloy3', 'Carbon'), ('Alloy3', 'Nickel'), ('Alloy3', 'Chromium'), ('Alloy3', 'TensileStrength'), ('Scrap1', 'Carbon'), ('Scrap1', 'Nickel'), ('Scrap1', 'Chromium'), ('Scrap1', 'TensileStrength'), ('Scrap2', 'Carbon'), ('Scrap2', 'Nickel'), ('Scrap2', 'Chromium'), ('Scrap2', 'Tens

<hr style="height:1.5px;background-color:gray;">

## Solve the model

In [18]:
opt = pyo.SolverFactory('glpk')

In [19]:
results = opt.solve(model, tee=True)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /var/folders/5k/rxb0jk152pb3hcrczw45mrm40000gn/T/tmpl_iwn6ya.glpk.raw
 --wglp /var/folders/5k/rxb0jk152pb3hcrczw45mrm40000gn/T/tmpimtidmtv.glpk.glp
 --cpxlp /var/folders/5k/rxb0jk152pb3hcrczw45mrm40000gn/T/tmpugi86f6x.pyomo.lp
Reading problem data from '/var/folders/5k/rxb0jk152pb3hcrczw45mrm40000gn/T/tmpugi86f6x.pyomo.lp'...
15 rows, 6 columns, 50 non-zeros
113 lines were read
Writing problem data to '/var/folders/5k/rxb0jk152pb3hcrczw45mrm40000gn/T/tmpimtidmtv.glpk.glp'...
94 lines were written
GLPK Simplex Optimizer 5.0
15 rows, 6 columns, 50 non-zeros
Preprocessing...
8 rows, 5 columns, 39 non-zeros
Scaling...
 A: min|aij| =  1.000e-03  max|aij| =  7.000e+04  ratio =  7.000e+07
GM: min|aij| =  2.474e-01  max|aij| =  4.041e+00  ratio =  1.633e+01
EQ: min|aij| =  6.507e-02  max|aij| =  1.000e+00  ratio =  1.537e+01
Constructing initial basis...
Size of triangular part is 8
      0: obj =   2.000000000

In [20]:
print(f'Termination condition: {results.solver.termination_condition}')
print(f'Optimal value: {pyo.value(model.total_cost):.4f}')
print(f'Optimal solution:')

for i in model.R:
    print(f'  x[{i}] = {pyo.value(model.x[i]):.4f}')

Termination condition: optimal
Optimal value: 7086.0768
Optimal solution:
  x[Alloy1] = 16.3923
  x[Alloy2] = 18.4911
  x[Alloy3] = 10.9328
  x[Scrap1] = 30.0000
  x[Scrap2] = 24.1838
